# Get inflation data
We want to get inflation data to use as a dimension in the model, to enable accounting for that in over time measures.
This is available via ONS but we need to find it; and in particular, the specific file we want.

In [ ]:
#set root
import sys
import os

# Get the project root (one level up from notebooks/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

'c:\\Users\\sj645\\git-repos\\ashe'

In [3]:
import pandas as pd
import requests

from get_data.initial_api_extraction import *

## Start exploration

In [5]:
#base url
endpoint = "https://api.beta.ons.gov.uk/v1"

#get datasets

dataset_resp = requests.get(endpoint + "/datasets")
dataset_json = dataset_resp.json()

dataset_df = pd.DataFrame(dataset_json)

#re-run but return all records
params = {"limit": 337}

dataset_resp = requests.get(endpoint + "/datasets", params = params)
dataset_json = dataset_resp.json()

#final result
items_df = pd.DataFrame(dataset_json["items"])

In [13]:
#see if we can identify anything to do with infltion
inflation_df = items_df.explode("keywords")
inflation_df = items_df[
    (items_df["keywords"].str.contains("inflation|cpi|index", case = False, na = False)) | 
    (items_df["title"].str.contains("cpi|inflation|index", case = False, na = False))
    ]

inflation_df

,contacts,description,keywords,id,last_updated,links,methodologies,national_statistic,next_release,qmi,...,title,unit_of_measure,type,publications,license,is_based_on,canonical_topic,subtopics,survey,related_content
14,[{'email': 'retail.sales.enquiries@ons.gov.uk'...,Value and volume of retail sales broken down b...,"[RSI, internet sales,goods bought,buying,spend...",retail-sales-index-large-and-small-businesses,2025-06-20T11:30:12.9Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,False,23 May 2025,{'href': 'https://www.ons.gov.uk/businessindus...,...,Retail sales index - large and small businesses,2019=100,filterable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,[{'email': 'retail.sales.enquiries@ons.gov.uk'...,Value and volume of retail sales for all busin...,"[RSI, internet sales,goods bought,buying,spend...",retail-sales-index-all-businesses,2025-06-20T11:38:18.485Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,False,23 May 2025,{'href': 'https://www.ons.gov.uk/businessindus...,...,Retail sales index - all businesses,2019=100,filterable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,[{'email': 'retail.sales.enquiries@ons.gov.uk'...,Retail sales data for Great Britain in value a...,"[RSI, internet sales,goods bought,buying,spend...",retail-sales-index,2025-06-20T11:41:50.657Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,False,23 May 2025,{'href': 'https://www.ons.gov.uk/businessindus...,...,Retail sales index,2019=100,filterable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,"[{'email': 'hpi@ons.gov.uk', 'name': 'Ceri Lew...",An experimental price index tracking the price...,"[private renting, regions,IPHRP,rent,inflation]",index-private-housing-rental-prices,2024-02-14T09:54:29.063Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,False,TBC,{'href': 'https://www.ons.gov.uk/economy/infla...,...,Index of Private Housing Rental Prices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,"[{'email': 'gdp@ons.gov.uk', 'name': 'Gross Do...",Gross domestic product (GDP) measures the valu...,"[GDP, economy]",gdp-to-four-decimal-places,2025-06-12T09:11:17.762Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,False,11 July 2025,{'href': 'https://www.ons.gov.uk/economy/gross...,...,GDP monthly estimate (incorporating the Index ...,Index. Seasonally adjusted 2016=100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,"[{'email': 'cpi@ons.gov.uk', 'name': 'Consumer...",CPIH is the most comprehensive measure of infl...,[Inflation],cpih01,2025-06-18T08:24:30.937Z,{'editions': {'href': 'https://api.beta.ons.go...,[{'href': 'https://www.ons.gov.uk/economy/infl...,True,To be announced,{'href': 'https://www.ons.gov.uk/economy/infla...,...,Consumer Prices Index including owner occupier...,Index: 2015=100,NaN,[{'href': 'https://www.ons.gov.uk/economy/infl...,Open Government Licence v3.0,NaN,NaN,NaN,NaN,NaN


## CPIH
Looks like CPIH is our only bet

In [ ]:
dataset_id = "cpih01"

cpih_df = inflation_df[inflation_df["id"] == dataset_id]

links = cpih_df["links"].apply(pd.Series) #turn links dict into a df

links_df = pd.concat([cpih_df.drop(columns = "links"), links], axis = 1) #replace links dict with above df

links

,editions,latest_version,self,taxonomy
37,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/econo...


In [18]:
#what if we check latest version?
url = links["latest_version"].tolist()[0]["href"]
url

'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/59'

In [20]:
#query 
cpih_resp = requests.get(url)

In [22]:
cpih_resp.json()

{'alerts': [],
 'collection_id': 'cmdcpih-70aa9c0161d612bc8260402b96e092924bf8e75291d5c8c4b470be7668c9373a',
 'dimensions': [{'label': 'Time',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mmm-yy',
   'id': 'mmm-yy',
   'name': 'time'},
  {'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only',
   'id': 'uk-only',
   'name': 'geography'},
  {'label': 'Aggregate',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/cpih1dim1aggid',
   'id': 'cpih1dim1aggid',
   'name': 'aggregate'}],
 'downloads': {'xls': {'href': 'https://download.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/59.xlsx',
   'size': '280307'},
  'csv': {'href': 'https://download.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/59.csv',
   'size': '4619579'},
  'csv

In [77]:
#think you want to download at this point...
csv_url = cpih_resp.json().get("downloads").get("csv").get("href")
csv_url

'https://download.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/59.csv'